#  Agentic RAG System with LangChain, LangGraph, FAISS & MCP

## Overview
This notebook implements a complete agentic RAG (Retrieval-Augmented Generation) system with flexible LLM options.

### Components:
1. **FAISS Vector Store**: Fast, efficient vector database for semantic search
2. **LangChain**: Framework for LLM applications
3. **LangGraph**: For agentic workflows with state management
4. **LLM Options**: OpenAI GPT-4 or Local Ollama (Llama 3)
5. **MCP (Model Context Protocol)**: For tool integration
6. **Embeddings**: OpenAI or HuggingFace (local)
7. **PyMuPDF**: Fast, reliable PDF processing

### What This System Does:
- Creates an autonomous agent that can:
  - Search documents using semantic similarity
  - Reason about retrieved information
  - Answer questions with citations
  - Use tools via MCP protocol
  - Maintain conversation state


### Why FAISS + PyMuPDF?
- **FAISS**: Facebook's industry-standard vector database - fast, memory-efficient, scalable
- **PyMuPDF (fitz)**: 3-5x faster than pypdf, better text extraction, handles complex PDFs

## Step 1: Installation and Imports

Installing all required packages. Works with both OpenAI and local Ollama.

In [1]:
# Install required packages
# Run this cell first to install all dependencies

!pip install -q langchain langchain-community langchain-core
!pip install -q langchain-openai  # For OpenAI models
!pip install -q langgraph
!pip install -q faiss-cpu  # FAISS vector database (CPU version)
!pip install -q sentence-transformers  # For local embeddings
!pip install -q ollama  # Python client for Ollama (optional if using OpenAI)
!pip install -q openai  # OpenAI Python client
!pip install -q tiktoken  # For OpenAI token counting
!pip install -q pymupdf  # For PDF document loading (better than pypdf)
!pip install -q beautifulsoup4  # For HTML parsing
!pip install -q mcp  # Model Context Protocol

print(" All packages installed successfully!")
print("\n Key packages:")
print("  - FAISS: Vector database for semantic search")
print("  - PyMuPDF: Fast and reliable PDF processing")
print("  - LangChain + LangGraph: Agentic framework")
print("  - OpenAI + Ollama: Flexible LLM options")

 All packages installed successfully!

 Key packages:
  - FAISS: Vector database for semantic search
  - PyMuPDF: Fast and reliable PDF processing
  - LangChain + LangGraph: Agentic framework
  - OpenAI + Ollama: Flexible LLM options


In [16]:
# Import all necessary libraries
import os
import json
from typing import TypedDict, Annotated, List, Dict, Any
from operator import add

# LangChain imports
from langchain_community.llms import Ollama
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools import Tool

# LangGraph imports for agentic workflows
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolExecutor, ToolInvocation
from langgraph.checkpoint.memory import MemorySaver

# Standard library imports
from datetime import datetime
import os
from dotenv import load_dotenv
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
load_dotenv()

print(" All imports successful!")

 All imports successful!


## Step 2: Initialize LLM and Embeddings

**Choose your LLM provider:**
- OpenAI (GPT-4, GPT-3.5) - Best quality, requires API key


In [17]:
# ========================================
# CONFIGURATION: Choose the LLM provider
# ========================================



print("Initializing OpenAI models...")

# Get API key from environment variable
# Make sure you've set: export OPENAI_API_KEY='your-key'
# Or set it directly here (not recommended for production)
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    print("  Warning: OPENAI_API_KEY not found in environment variables")
    print("Please set it with: export OPENAI_API_KEY='your-key'")
    print("Or uncomment the line below and add your key directly:")
    # openai_api_key = "sk-..."  # Uncomment and add your key

# Initialize OpenAI LLM
llm = ChatOpenAI(
    model="gpt-4o",  # Options: gpt-4o, gpt-4o-mini, gpt-4-turbo, gpt-3.5-turbo
    temperature=0.7,        # Controls randomness (0=deterministic, 1=creative)
    api_key=openai_api_key,
    max_tokens=2000         # Maximum response length
)

# Test the LLM
test_response = llm.invoke([HumanMessage(content="Say 'Hello, I am using OpenAI!'")])
print(f"\n LLM Test Response: {test_response.content}\n")

# Initialize OpenAI embeddings (faster and higher quality)
print("Initializing OpenAI embeddings...")
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # Options: text-embedding-3-small, text-embedding-3-large
    api_key=openai_api_key
)

# Test embeddings
test_embedding = embeddings.embed_query("This is a test sentence.")
print(f" Embedding dimension: {len(test_embedding)}")
print(f" First 5 values: {test_embedding[:5]}")
print("\n OpenAI LLM and embeddings initialized successfully!")



# ========================================
# Summary
# ========================================
print("\n" + "="*60)

print(" Using OpenAI GPT-4 + OpenAI Embeddings")
print("   Benefits: Best quality, fast, reliable")
print("   Cost: Pay per token usage")

print("="*60)

Initializing OpenAI models...

 LLM Test Response: Hello, I am using OpenAI!

Initializing OpenAI embeddings...
 Embedding dimension: 1536
 First 5 values: [0.024632183834910393, -1.6649944882374257e-05, -0.013136658817529678, -0.007176156155765057, -0.02695712260901928]

 OpenAI LLM and embeddings initialized successfully!

 Using OpenAI GPT-4 + OpenAI Embeddings
   Benefits: Best quality, fast, reliable
   Cost: Pay per token usage


## Step 3: Create Sample Knowledge Base

We'll create sample documents about AI and ML topics for our RAG system.
In production, you would load documents from PDFs using PyMuPDF, websites, or databases.

**PyMuPDF Benefits:**
- 3-5x faster than pypdf
- Better text extraction quality
- Handles complex PDFs (forms, tables, images)
- Reliable with large documents

In [18]:
# Create sample documents for our knowledge base
# These simulate real documents that would be loaded from files

sample_documents = [
    {
        "content": """Artificial Intelligence (AI) agents are autonomous systems that perceive their 
        environment, make decisions, and take actions to achieve specific goals. Modern AI agents 
        use Large Language Models (LLMs) as their reasoning engine, combined with tools for 
        external actions. Key characteristics include autonomy, goal-directed behavior, perception, 
        action capability, and reasoning.""",
        "metadata": {"source": "ai_fundamentals.txt", "topic": "AI Agents", "date": "2024-01-15"}
    },
    {
        "content": """Retrieval-Augmented Generation (RAG) is a technique that enhances LLM responses 
        by retrieving relevant information from external knowledge bases. The process involves: 
        1) Converting documents into embeddings, 2) Storing embeddings in a vector database, 
        3) Converting user queries into embeddings, 4) Finding similar documents using semantic search, 
        5) Augmenting the LLM prompt with retrieved context. This reduces hallucinations and provides 
        up-to-date information.""",
        "metadata": {"source": "rag_systems.txt", "topic": "RAG", "date": "2024-02-20"}
    },
    {
        "content": """FAISS (Facebook AI Similarity Search) is a library for efficient similarity search 
        and clustering of dense vectors. It's optimized for large-scale vector search and supports 
        various index types including flat (exact search), IVF (inverted file index), and HNSW 
        (Hierarchical Navigable Small World). FAISS is particularly useful for RAG systems where 
        fast retrieval of similar documents is crucial.""",
        "metadata": {"source": "vector_databases.txt", "topic": "FAISS", "date": "2024-03-10"}
    },
    {
        "content": """LangGraph is a framework for building stateful, multi-actor applications with LLMs. 
        It extends LangChain by providing graph-based orchestration where nodes represent computation 
        steps and edges represent transitions. Key features include: state management across nodes, 
        conditional branching, cycles for iterative processing, human-in-the-loop patterns, and 
        checkpointing for persistence. LangGraph is ideal for complex agentic workflows.""",
        "metadata": {"source": "langgraph_guide.txt", "topic": "LangGraph", "date": "2024-04-05"}
    },
    {
        "content": """The Model Context Protocol (MCP) is an open standard for connecting AI agents 
        with external tools and data sources. MCP servers expose capabilities through a standardized 
        interface including: tools (functions the agent can call), resources (data the agent can access), 
        and prompts (reusable templates). This allows agents to interact with databases, APIs, file systems, 
        and other services in a consistent way.""",
        "metadata": {"source": "mcp_protocol.txt", "topic": "MCP", "date": "2024-05-12"}
    },
    {
        "content": """Embeddings are dense vector representations of text that capture semantic meaning. 
        Similar texts have similar embeddings (measured by cosine similarity or dot product). 
        Popular embedding models include OpenAI's text-embedding-ada-002, sentence-transformers 
        from HuggingFace, and specialized models like instructor embeddings. Good embeddings are 
        crucial for RAG system performance as they determine retrieval quality.""",
        "metadata": {"source": "embeddings_explained.txt", "topic": "Embeddings", "date": "2024-06-18"}
    },
    {
        "content": """Tool use in AI agents involves the ability to call external functions or APIs to 
        extend capabilities beyond text generation. The process typically involves: 1) Tool description 
        (schema defining inputs/outputs), 2) Tool selection (agent decides which tool to use), 
        3) Parameter extraction (agent generates proper arguments), 4) Execution (framework calls the tool), 
        5) Result integration (agent processes the output). This enables agents to perform calculations, 
        search the web, query databases, and more.""",
        "metadata": {"source": "tool_usage.txt", "topic": "Tools", "date": "2024-07-22"}
    }
]

# Convert to LangChain Document objects
documents = [
    Document(page_content=doc["content"], metadata=doc["metadata"]) 
    for doc in sample_documents
]

print(f" Created {len(documents)} sample documents")
print(f"\n Sample document preview:")
print(f"Content: {documents[0].page_content[:150]}...")
print(f"Metadata: {documents[0].metadata}")

 Created 7 sample documents

 Sample document preview:
Content: Artificial Intelligence (AI) agents are autonomous systems that perceive their 
        environment, make decisions, and take actions to achieve speci...
Metadata: {'source': 'ai_fundamentals.txt', 'topic': 'AI Agents', 'date': '2024-01-15'}


In [19]:
# ==========================================
# BONUS: How to Load PDFs with PyMuPDF
# ==========================================

import fitz  # PyMuPDF

def load_pdf_with_pymupdf(pdf_path: str) -> List[Document]:
    """
    Load PDF using PyMuPDF (faster and better than pypdf).
    
    Args:
        pdf_path: Path to PDF file
    
    Returns:
        List of Document objects with text and metadata
    """
    documents = []
    
    try:
        # Open PDF
        pdf_document = fitz.open(pdf_path)
        
        # Extract text from each page
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            text = page.get_text()
            
            # Skip empty pages
            if text.strip():
                # Create document with metadata
                doc = Document(
                    page_content=text,
                    metadata={
                        "source": pdf_path,
                        "page": page_num + 1,
                        "total_pages": len(pdf_document)
                    }
                )
                documents.append(doc)
        
        pdf_document.close()
        print(f" Loaded {len(documents)} pages from {pdf_path}")
        
    except Exception as e:
        print(f" Error loading PDF: {str(e)}")
        return []
    
    return documents

# Example usage (uncomment to use):
# documents = load_pdf_with_pymupdf("your_document.pdf")

print("\n To load your own PDFs:")
print("  1. Place your PDF in the same directory")
print("  2. Uncomment: documents = load_pdf_with_pymupdf('your_file.pdf')")
print("  3. Run this cell to load the PDF")


 To load your own PDFs:
  1. Place your PDF in the same directory
  2. Uncomment: documents = load_pdf_with_pymupdf('your_file.pdf')
  3. Run this cell to load the PDF


## Step 4: Text Splitting and Chunking

Split documents into smaller chunks for better retrieval.
Smaller chunks = more precise retrieval but might lose context.
Larger chunks = more context but less precise.

In [20]:
# Initialize text splitter
# RecursiveCharacterTextSplitter tries to keep semantically related text together
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Maximum characters per chunk
    chunk_overlap=50,      # Overlap between chunks to maintain context
    length_function=len,   # Function to measure chunk size
    separators=["\n\n", "\n", ". ", " ", ""]  # Try these separators in order
)

# Split documents into chunks
split_documents = text_splitter.split_documents(documents)

print(f" Split {len(documents)} documents into {len(split_documents)} chunks")
print(f"\n Chunk statistics:")
chunk_lengths = [len(doc.page_content) for doc in split_documents]
print(f"  - Average chunk size: {sum(chunk_lengths) / len(chunk_lengths):.0f} characters")
print(f"  - Min chunk size: {min(chunk_lengths)} characters")
print(f"  - Max chunk size: {max(chunk_lengths)} characters")

# Show a sample chunk
print(f"\n Sample chunk:")
print(f"Content: {split_documents[0].page_content}")
print(f"Metadata: {split_documents[0].metadata}")

 Split 7 documents into 9 chunks

 Chunk statistics:
  - Average chunk size: 362 characters
  - Min chunk size: 23 characters
  - Max chunk size: 497 characters

 Sample chunk:
Content: Artificial Intelligence (AI) agents are autonomous systems that perceive their 
        environment, make decisions, and take actions to achieve specific goals. Modern AI agents 
        use Large Language Models (LLMs) as their reasoning engine, combined with tools for 
        external actions. Key characteristics include autonomy, goal-directed behavior, perception, 
        action capability, and reasoning.
Metadata: {'source': 'ai_fundamentals.txt', 'topic': 'AI Agents', 'date': '2024-01-15'}


##  Step 5: Create FAISS Vector Store

Build the vector database for semantic search using FAISS (Facebook AI Similarity Search).

### Why FAISS?
- **Speed**: 10-100x faster than basic vector search
- **Memory Efficient**: Optimized for large-scale datasets
- **Industry Standard**: Used by Facebook, Google, Microsoft
- **Scalable**: Handles millions of vectors efficiently
- **Flexible**: Multiple index types (Flat, IVF, HNSW) for different use cases

FAISS indexes document embeddings for lightning-fast similarity search.

In [21]:
# Create FAISS vector store from documents
# This will:
# 1. Generate embeddings for all document chunks
# 2. Build a FAISS index for fast similarity search
# 3. Store document metadata for retrieval

print("Creating FAISS vector store...")
print("(This may take a minute for embedding generation)\n")

vectorstore = FAISS.from_documents(
    documents=split_documents,
    embedding=embeddings
)

print(" FAISS vector store created successfully!")
print(f"\n Index Statistics:")
print(f"  - Total vectors indexed: {vectorstore.index.ntotal}")
print(f"  - Vector dimension: {vectorstore.index.d}")
print(f"  - Index type: {type(vectorstore.index).__name__}")

# Test the vector store with a sample query
test_query = "What is RAG?"
print(f"\n Test query: '{test_query}'")

# Perform similarity search
test_results = vectorstore.similarity_search(
    query=test_query,
    k=2  # Return top 2 most similar documents
)

print(f"\n Top {len(test_results)} results:")
for i, doc in enumerate(test_results, 1):
    print(f"\n{i}. Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"   Topic: {doc.metadata.get('topic', 'Unknown')}")
    print(f"   Content preview: {doc.page_content[:150]}...")

# Save the vector store locally (optional but recommended)
vectorstore.save_local("faiss_index")
print("\n Vector store saved to './faiss_index'")

print("\n FAISS Performance Tips:")
print("  - Flat index (default): Best for <1M vectors, exact search")
print("  - IVF index: Better for 1M-10M vectors, approximate search")
print("  - HNSW index: Best for 10M+ vectors, fastest approximate search")
print("  - Current setup uses Flat index (exact, reliable)")

Creating FAISS vector store...
(This may take a minute for embedding generation)

 FAISS vector store created successfully!

 Index Statistics:
  - Total vectors indexed: 9
  - Vector dimension: 1536
  - Index type: IndexFlatL2

 Test query: 'What is RAG?'

 Top 2 results:

1. Source: rag_systems.txt
   Topic: RAG
   Content preview: Retrieval-Augmented Generation (RAG) is a technique that enhances LLM responses 
        by retrieving relevant information from external knowledge ba...

2. Source: vector_databases.txt
   Topic: FAISS
   Content preview: FAISS (Facebook AI Similarity Search) is a library for efficient similarity search 
        and clustering of dense vectors. It's optimized for large-...

 Vector store saved to './faiss_index'

 FAISS Performance Tips:
  - Flat index (default): Best for <1M vectors, exact search
  - IVF index: Better for 1M-10M vectors, approximate search
  - HNSW index: Best for 10M+ vectors, fastest approximate search
  - Current setup uses Flat index

## Step 6: Create RAG Tools for the Agent

Define tools that our agent can use:
1. **search_documents**: Semantic search in the vector store
2. **get_current_time**: Get current timestamp
3. **calculator**: Perform basic calculations

These tools follow the MCP-inspired pattern for tool definition.

In [22]:
# Define the document search tool
def search_documents(query: str, k: int = 3) -> str:
    """
    Search the document knowledge base for relevant information.
    
    Args:
        query: The search query
        k: Number of results to return (default: 3)
    
    Returns:
        Formatted string with search results and metadata
    """
    # Perform similarity search
    results = vectorstore.similarity_search(query, k=k)
    
    # Format results with source attribution
    formatted_results = []
    for i, doc in enumerate(results, 1):
        formatted_results.append(
            f"Result {i}:\n"
            f"Source: {doc.metadata.get('source', 'Unknown')}\n"
            f"Topic: {doc.metadata.get('topic', 'Unknown')}\n"
            f"Content: {doc.page_content}\n"
        )
    
    return "\n---\n".join(formatted_results)

# Define a time tool
def get_current_time() -> str:
    """
    Get the current date and time.
    
    Returns:
        Current timestamp as a formatted string
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Define a simple calculator tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression.
    
    Args:
        expression: Mathematical expression to evaluate (e.g., "2 + 2")
    
    Returns:
        Result of the calculation
    """
    try:
        # Safe evaluation of basic math expressions
        result = eval(expression, {"__builtins__": {}}, {})
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Wrap functions as LangChain Tools
# Tools need a name, description, and function
tools = [
    Tool(
        name="search_documents",
        func=search_documents,
        description="""Search the document knowledge base for information about AI, ML, RAG systems, 
        LangGraph, FAISS, embeddings, and related topics. Use this when you need to find specific 
        information from the knowledge base. Input should be a search query string."""
    ),
    Tool(
        name="get_current_time",
        func=get_current_time,
        description="Get the current date and time. Use this when asked about the current time or date."
    ),
    Tool(
        name="calculator",
        func=calculator,
        description="""Perform mathematical calculations. Input should be a mathematical expression 
        as a string (e.g., '2 + 2' or '10 * 5'). Supports basic arithmetic operations."""
    )
]

print(" Created 3 tools for the agent:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description[:80]}...")

# Test the tools
print("\n Testing tools:")
print(f"\n1. search_documents('embeddings'):\n{search_documents('embeddings', k=1)[:200]}...")
print(f"\n2. get_current_time():\n{get_current_time()}")
print(f"\n3. calculator('2 + 2'):\n{calculator('2 + 2')}")

 Created 3 tools for the agent:
  - search_documents: Search the document knowledge base for information about AI, ML, RAG systems, 
 ...
  - get_current_time: Get the current date and time. Use this when asked about the current time or dat...
  - calculator: Perform mathematical calculations. Input should be a mathematical expression 
  ...

 Testing tools:

1. search_documents('embeddings'):
Result 1:
Source: embeddings_explained.txt
Topic: Embeddings
Content: Embeddings are dense vector representations of text that capture semantic meaning. 
        Similar texts have similar embeddings ...

2. get_current_time():
2025-11-10 12:00:05

3. calculator('2 + 2'):
Result: 4


##  Step 7: Define Agent State

LangGraph uses a state object that flows through the graph.
The state maintains conversation history, intermediate results, and metadata.

In [23]:
# Define the state structure for our agent
# TypedDict provides type hints for the state object
class AgentState(TypedDict):
    """
    State object that flows through the agent graph.
    
    Attributes:
        messages: Conversation history (human and AI messages)
        tool_calls: History of tool invocations
        iterations: Number of reasoning loops completed
        final_answer: The agent's final response
    """
    # The messages list accumulates the conversation
    # Annotated with 'add' operator to append new messages
    messages: Annotated[List[Any], add]
    
    # Track tool usage for observability
    tool_calls: Annotated[List[Dict], add]
    
    # Count iterations to prevent infinite loops
    iterations: int
    
    # Final answer to return to the user
    final_answer: str

print(" Agent state structure defined")
print("\n📋 State fields:")
print("  - messages: Conversation history")
print("  - tool_calls: Tool usage tracking")
print("  - iterations: Loop counter")
print("  - final_answer: Final response")

 Agent state structure defined

📋 State fields:
  - messages: Conversation history
  - tool_calls: Tool usage tracking
  - iterations: Loop counter
  - final_answer: Final response


## Step 8: Create Agent Reasoning Functions

These are the core functions that define agent behavior:
1. **agent_node**: The reasoning step - decides what to do next
2. **tool_node**: Executes tools selected by the agent
3. **should_continue**: Decides whether to continue or finish

This implements the ReAct (Reasoning + Acting) pattern.

In [24]:
# Maximum iterations to prevent infinite loops
MAX_ITERATIONS = 5

def agent_node(state: AgentState) -> AgentState:
    """
    The reasoning node where the agent thinks and decides what to do.
    
    This function:
    1. Takes current state (conversation history)
    2. Prompts the LLM to reason about next action
    3. Decides whether to use a tool or provide an answer
    4. Updates state with the decision
    
    Args:
        state: Current agent state
    
    Returns:
        Updated state with agent's reasoning
    """
    messages = state["messages"]
    iterations = state.get("iterations", 0)
    
    # Create a prompt that explains available tools
    tool_descriptions = "\n".join([
        f"- {tool.name}: {tool.description}" 
        for tool in tools
    ])
    
    system_prompt = f"""You are a helpful AI assistant with access to tools. 
    
Available tools:
{tool_descriptions}

To use a tool, respond with:
TOOL: tool_name
INPUT: input_for_tool

If you have enough information to answer, respond with:
ANSWER: your_final_answer

Think step by step. Use tools when you need specific information.
Current iteration: {iterations}/{MAX_ITERATIONS}
"""
    
    # Get LLM response
    # OpenAI uses chat messages format
    full_messages = [SystemMessage(content=system_prompt)] + messages
    response_obj = llm.invoke(full_messages)
    response = response_obj.content  # Extract text from ChatOpenAI response

    
    # Update state with agent's response
    state["messages"].append(AIMessage(content=response))
    state["iterations"] = iterations + 1
    
    # Parse the response to determine next action
    if "ANSWER:" in response:
        # Extract final answer
        answer = response.split("ANSWER:")[1].strip()
        state["final_answer"] = answer
    
    return state

def tool_node(state: AgentState) -> AgentState:
    """
    Execute tools based on agent's decision.
    
    This function:
    1. Parses the agent's response for tool calls
    2. Executes the requested tool
    3. Adds tool results back to the conversation
    
    Args:
        state: Current agent state
    
    Returns:
        Updated state with tool results
    """
    messages = state["messages"]
    last_message = messages[-1].content
    
    # Parse tool call from agent's response
    if "TOOL:" in last_message and "INPUT:" in last_message:
        # Extract tool name and input
        tool_name = last_message.split("TOOL:")[1].split("\n")[0].strip()
        tool_input = last_message.split("INPUT:")[1].strip()
        
        # Find and execute the tool
        tool = next((t for t in tools if t.name == tool_name), None)
        
        if tool:
            try:
                # Execute the tool
                result = tool.func(tool_input)
                
                # Record tool usage
                state["tool_calls"].append({
                    "tool": tool_name,
                    "input": tool_input,
                    "output": result
                })
                
                # Add tool result to conversation
                state["messages"].append(
                    HumanMessage(content=f"Tool Result ({tool_name}):\n{result}")
                )
            except Exception as e:
                state["messages"].append(
                    HumanMessage(content=f"Tool Error: {str(e)}")
                )
        else:
            state["messages"].append(
                HumanMessage(content=f"Error: Tool '{tool_name}' not found")
            )
    
    return state

def should_continue(state: AgentState) -> str:
    """
    Decide whether to continue the agent loop or finish.
    
    Stops if:
    - Agent provided a final answer
    - Maximum iterations reached
    
    Args:
        state: Current agent state
    
    Returns:
        Next node to execute: "tools", "agent", or "end"
    """
    messages = state["messages"]
    iterations = state.get("iterations", 0)
    last_message = messages[-1].content if messages else ""
    
    # Stop if we have a final answer
    if "ANSWER:" in last_message or state.get("final_answer"):
        return "end"
    
    # Stop if max iterations reached
    if iterations >= MAX_ITERATIONS:
        state["final_answer"] = "Maximum iterations reached. Unable to complete task."
        return "end"
    
    # If agent wants to use a tool, go to tool node
    if "TOOL:" in last_message:
        return "tools"
    
    # Otherwise, continue reasoning
    return "agent"

print(" Agent reasoning functions defined")
print("\n🔄 Agent loop logic:")
print("  1. agent_node: Reason about what to do")
print("  2. should_continue: Decide next action")
print("  3. tool_node: Execute tools if needed")
print("  4. Repeat until answer or max iterations")

 Agent reasoning functions defined

🔄 Agent loop logic:
  1. agent_node: Reason about what to do
  2. should_continue: Decide next action
  3. tool_node: Execute tools if needed
  4. Repeat until answer or max iterations


## Step 9: Build the LangGraph Agent

Now we assemble everything into a graph structure:
- Nodes: agent_node, tool_node
- Edges: Conditional routing based on should_continue
- State: Flows through the graph and accumulates results

In [25]:
# Create the state graph
workflow = StateGraph(AgentState)

# Add nodes to the graph
# Each node is a function that processes and updates state
workflow.add_node("agent", agent_node)  # Reasoning node
workflow.add_node("tools", tool_node)    # Tool execution node

# Set the entry point - where the graph starts
workflow.set_entry_point("agent")

# Add conditional edges
# After each node, should_continue decides the next step
workflow.add_conditional_edges(
    "agent",           # From agent node
    should_continue,   # Use this function to decide
    {
        "tools": "tools",  # If "tools", go to tools node
        "agent": "agent",  # If "agent", loop back to agent node
        "end": END         # If "end", finish the graph
    }
)

# After tool execution, always go back to agent for reasoning
workflow.add_edge("tools", "agent")

# Add memory for state persistence
# This allows the agent to remember across multiple runs
memory = MemorySaver()

# Compile the graph into a runnable agent
agent = workflow.compile(checkpointer=memory)

print(" LangGraph agent compiled successfully!")
print("\n Agent graph structure:")
print("  START → agent → [should_continue] → tools → agent → ... → END")
print("\n🔄 Workflow:")
print("  1. User query enters at 'agent' node")
print("  2. Agent reasons and decides action")
print("  3. If tool needed: execute and return to agent")
print("  4. If answer ready: proceed to END")
print("  5. State persists across iterations")

 LangGraph agent compiled successfully!

 Agent graph structure:
  START → agent → [should_continue] → tools → agent → ... → END

🔄 Workflow:
  1. User query enters at 'agent' node
  2. Agent reasons and decides action
  3. If tool needed: execute and return to agent
  4. If answer ready: proceed to END
  5. State persists across iterations


## Step 10: Helper Function to Run Agent

Create a convenient function to interact with our agent.

In [26]:
def run_agent(query: str, thread_id: str = "default", verbose: bool = True) -> str:
    """
    Run the agent with a user query.
    
    Args:
        query: User's question or request
        thread_id: Conversation thread identifier for state persistence
        verbose: Whether to print intermediate steps
    
    Returns:
        Final answer from the agent
    """
    # Initialize state with user query
    initial_state = {
        "messages": [HumanMessage(content=query)],
        "tool_calls": [],
        "iterations": 0,
        "final_answer": ""
    }
    
    # Configuration for state persistence
    config = {"configurable": {"thread_id": thread_id}}
    
    if verbose:
        print("\n" + "="*80)
        print(f" AGENT EXECUTION: {query}")
        print("="*80 + "\n")
    
    # Run the agent graph
    result = agent.invoke(initial_state, config)
    
    # Display execution trace if verbose
    if verbose:
        print("\n📋 EXECUTION TRACE:")
        print(f"Total iterations: {result['iterations']}")
        print(f"Tools used: {len(result['tool_calls'])}")
        
        if result['tool_calls']:
            print("\n🔧 Tool Calls:")
            for i, call in enumerate(result['tool_calls'], 1):
                print(f"  {i}. {call['tool']}")
                print(f"     Input: {call['input'][:100]}...")
                print(f"     Output: {call['output'][:100]}...\n")
    
    # Extract and return final answer
    final_answer = result.get("final_answer", "No answer generated")
    
    if verbose:
        print("\n" + "="*80)
        print(" FINAL ANSWER:")
        print("="*80)
        print(final_answer)
        print("\n" + "="*80 + "\n")
    
    return final_answer

print(" Helper function created")
print("\n📞 Usage: run_agent('your question here')")

 Helper function created

📞 Usage: run_agent('your question here')


## Step 11: Test the Agent

Let's test our agentic RAG system with various queries.

In [29]:
# Test 1: Simple RAG query
print("\n" + "#"*80)
print("TEST 1: Simple RAG Query")
print("#"*80)

answer1 = run_agent(
    "What is RAG and how does it work?",
    thread_id="test1"
)


################################################################################
TEST 1: Simple RAG Query
################################################################################

 AGENT EXECUTION: What is RAG and how does it work?



RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4o in organization org-AMaYCohKJjROv8W6tXaQsmgs on tokens per min (TPM): Limit 30000, Requested 55630. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [28]:
# Test 2: Multi-step query requiring tool use
print("\n" + "#"*80)
print("TEST 2: Multi-Step Query")
print("#"*80)

answer2 = run_agent(
    "Explain LangGraph and tell me what time it is right now.",
    thread_id="test2"
)


################################################################################
TEST 2: Multi-Step Query
################################################################################

 AGENT EXECUTION: Explain LangGraph and tell me what time it is right now.


📋 EXECUTION TRACE:
Total iterations: 2
Tools used: 4

🔧 Tool Calls:
  1. search_documents
     Input: LangGraph

Now, I'll get the current time.

TOOL: get_current_time...
     Output: Result 1:
Source: langgraph_guide.txt
Topic: LangGraph
Content: LangGraph is a framework for buildin...

  2. search_documents
     Input: LangGraph

Now, I'll get the current time.

TOOL: get_current_time...
     Output: Result 1:
Source: langgraph_guide.txt
Topic: LangGraph
Content: LangGraph is a framework for buildin...

  3. search_documents
     Input: LangGraph

Now, I'll get the current time.

TOOL: get_current_time...
     Output: Result 1:
Source: langgraph_guide.txt
Topic: LangGraph
Content: LangGraph is a framework for buildin...



In [ ]:
# Test 3: Query requiring multiple document lookups
print("\n" + "#"*80)
print("TEST 3: Multiple Document Lookups")
print("#"*80)

answer3 = run_agent(
    "Compare FAISS and embeddings. How do they work together?",
    thread_id="test3"
)

In [ ]:
# Test 4: Query combining RAG and calculation
print("\n" + "#"*80)
print("TEST 4: RAG + Calculation")
print("#"*80)

answer4 = run_agent(
    "If I have 7 documents about AI agents, and I split each into 3 chunks, how many chunks total? Also, what are AI agents?",
    thread_id="test4"
)

## Step 12: Interactive Agent Session

Create an interactive chat interface for continuous conversation.

In [ ]:
def interactive_agent_session():
    """
    Start an interactive session with the agent.
    Type 'quit' or 'exit' to end the session.
    """
    print("\n" + "="*80)
    print(" INTERACTIVE AGENT SESSION")
    print("="*80)
    print("\nWelcome! I'm your AI assistant with access to:")
    print("  - Document search (AI/ML knowledge base)")
    print("  - Current time lookup")
    print("  - Basic calculator")
    print("\nType 'quit' or 'exit' to end the session.")
    print("="*80 + "\n")
    
    thread_id = f"interactive_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    while True:
        try:
            # Get user input
            user_input = input("\n👤 You: ").strip()
            
            # Check for exit commands
            if user_input.lower() in ['quit', 'exit', 'bye']:
                print("\n Goodbye! Thanks for chatting.\n")
                break
            
            # Skip empty inputs
            if not user_input:
                continue
            
            # Run the agent
            answer = run_agent(user_input, thread_id=thread_id, verbose=False)
            print(f"\n Agent: {answer}")
            
        except KeyboardInterrupt:
            print("\n\n Session interrupted. Goodbye!\n")
            break
        except Exception as e:
            print(f"\n Error: {str(e)}")
            print("Please try again.\n")

# Uncomment to start interactive session
# interactive_agent_session()

print("\n Interactive session function ready")
print("\n To start chatting, uncomment and run:")
print("   interactive_agent_session()")

## Step 13: MCP Server Integration (Conceptual)

While full MCP implementation requires a running server, here's how to integrate MCP tools.
This demonstrates the pattern for production deployments.

In [ ]:
# MCP Tool Integration Pattern
# In production, you would connect to actual MCP servers

class MCPToolWrapper:
    """
    Wrapper for MCP (Model Context Protocol) tools.
    
    MCP provides a standardized way to expose tools to AI agents.
    This wrapper demonstrates how to integrate MCP tools into our agent.
    """
    
    def __init__(self, tool_name: str, tool_description: str, endpoint: str):
        """
        Initialize MCP tool wrapper.
        
        Args:
            tool_name: Name of the tool
            tool_description: Description for the agent
            endpoint: MCP server endpoint URL
        """
        self.name = tool_name
        self.description = tool_description
        self.endpoint = endpoint
    
    def call(self, input_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Call the MCP tool via HTTP/JSON-RPC.
        
        In production, this would make actual HTTP requests to MCP servers.
        
        Args:
            input_data: Tool input parameters
        
        Returns:
            Tool execution result
        """
        # Simulated MCP call (in production, use requests library)
        print(f"📡 [MCP] Calling {self.name} at {self.endpoint}")
        print(f"📡 [MCP] Input: {input_data}")
        
        # Example: actual implementation would look like:
        # import requests
        # response = requests.post(
        #     self.endpoint,
        #     json={
        #         "jsonrpc": "2.0",
        #         "method": "tools/call",
        #         "params": {
        #             "name": self.name,
        #             "arguments": input_data
        #         },
        #         "id": 1
        #     }
        # )
        # return response.json()["result"]
        
        return {
            "status": "success",
            "message": "MCP tool executed (simulated)"
        }

# Example MCP tool definitions
mcp_tools_config = [
    {
        "name": "filesystem_read",
        "description": "Read contents of a file from the filesystem",
        "endpoint": "http://localhost:3000/mcp/filesystem"
    },
    {
        "name": "database_query",
        "description": "Execute SQL queries on connected databases",
        "endpoint": "http://localhost:3000/mcp/database"
    },
    {
        "name": "web_scrape",
        "description": "Scrape content from web pages",
        "endpoint": "http://localhost:3000/mcp/web"
    }
]

# Create MCP tool wrappers
mcp_tools = [
    MCPToolWrapper(
        tool_name=config["name"],
        tool_description=config["description"],
        endpoint=config["endpoint"]
    )
    for config in mcp_tools_config
]

print(" MCP tool integration pattern demonstrated")
print("\n📡 MCP Tools (conceptual):")
for tool in mcp_tools:
    print(f"  - {tool.name}: {tool.description}")
    print(f"    Endpoint: {tool.endpoint}")

print("\n To use in production:")
print("  1. Start MCP server: mcp-server --port 3000")
print("  2. Implement actual HTTP calls in MCPToolWrapper.call()")
print("  3. Add MCP tools to the agent's tools list")
print("  4. Agent can now use MCP tools like any other tool")

##  Step 14: Agent Analytics and Observability

Monitor and analyze agent performance.

In [ ]:
def analyze_agent_performance(state: AgentState) -> Dict[str, Any]:
    """
    Analyze agent execution performance.
    
    Args:
        state: Final agent state after execution
    
    Returns:
        Performance metrics dictionary
    """
    metrics = {
        "total_iterations": state.get("iterations", 0),
        "total_messages": len(state.get("messages", [])),
        "tool_calls_count": len(state.get("tool_calls", [])),
        "tools_used": [],
        "success": bool(state.get("final_answer")),
        "answer_length": len(state.get("final_answer", ""))
    }
    
    # Analyze tool usage
    tool_usage = {}
    for call in state.get("tool_calls", []):
        tool_name = call["tool"]
        tool_usage[tool_name] = tool_usage.get(tool_name, 0) + 1
    
    metrics["tools_used"] = tool_usage
    
    return metrics

# Example: Run agent and analyze performance
print("\n PERFORMANCE ANALYSIS EXAMPLE")
print("="*80 + "\n")

# Run a test query
test_state = {
    "messages": [HumanMessage(content="What are embeddings?")],
    "tool_calls": [],
    "iterations": 0,
    "final_answer": ""
}

result_state = agent.invoke(test_state, {"configurable": {"thread_id": "analysis"}})
metrics = analyze_agent_performance(result_state)

print("📈 Performance Metrics:")
print(f"  - Total Iterations: {metrics['total_iterations']}")
print(f"  - Messages Exchanged: {metrics['total_messages']}")
print(f"  - Tool Calls: {metrics['tool_calls_count']}")
print(f"  - Tools Used: {metrics['tools_used']}")
print(f"  - Success: {metrics['success']}")
print(f"  - Answer Length: {metrics['answer_length']} characters")

print("\n Analytics system ready for production monitoring")

##  Step 15: Save and Load Vector Store

Persist your vector store for reuse across sessions.

In [ ]:
# Save the vector store to disk
def save_vectorstore(vectorstore, path: str = "faiss_index"):
    """
    Save FAISS vector store to disk.
    
    Args:
        vectorstore: FAISS vectorstore instance
        path: Directory path to save to
    """
    vectorstore.save_local(path)
    print(f" Vector store saved to '{path}'")

# Load the vector store from disk
def load_vectorstore(path: str = "faiss_index", embeddings_model=None):
    """
    Load FAISS vector store from disk.
    
    Args:
        path: Directory path to load from
        embeddings_model: Embeddings model (must match the one used to create)
    
    Returns:
        Loaded FAISS vectorstore
    """
    if embeddings_model is None:
        embeddings_model = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )
    
    vectorstore = FAISS.load_local(
        path, 
        embeddings_model,
        allow_dangerous_deserialization=True  # Required for pickle loading
    )
    print(f" Vector store loaded from '{path}'")
    return vectorstore

# Save current vector store
save_vectorstore(vectorstore, "faiss_index")

# Example: Load it back
# loaded_vectorstore = load_vectorstore("faiss_index", embeddings)

print("\n Usage:")
print("  - Save: save_vectorstore(vectorstore, 'path')")
print("  - Load: loaded_vs = load_vectorstore('path', embeddings)")

## Step 16: Summary and Next Steps

### What We Built:

1.  **Flexible LLM**: OpenAI GPT-4 or Local Ollama (Llama 3.2)
2.  **Flexible Embeddings**: OpenAI or HuggingFace sentence-transformers
3.  **FAISS Vector Database**: Industry-standard, fast, scalable semantic search
4.  **RAG System**: Document retrieval and augmented generation
5.  **Agentic Framework**: LangGraph with state management
6.  **Tool Integration**: Search, calculator, time tools
7.  **MCP Pattern**: Model Context Protocol integration pattern
8.  **PyMuPDF**: Fast, reliable PDF processing (3-5x faster than pypdf)
9.  **Observability**: Performance monitoring and analytics

### Key Features:

- **Flexible**: Use OpenAI for best quality or Ollama for free local operation
- **Fast**: FAISS provides lightning-fast vector search
- **Reliable**: PyMuPDF handles complex PDFs better than alternatives
- **Privacy Options**: Local option keeps all data on your machine
- **Extensible**: Easy to add new tools and documents
- **Production-Ready**: State management, error handling, monitoring
- **Open Source**: All components are FOSS

### Technology Choices:

**FAISS vs Other Vector Databases:**
-  10-100x faster than basic implementations
-  Memory efficient for large datasets
-  Industry battle-tested (Facebook, Google use it)
-  Multiple index types for different scales
-  No external server required (unlike Chroma, Pinecone)
-  Perfect for local development and production

**PyMuPDF vs pypdf:**
-  3-5x faster processing
-  Better text extraction quality
-  Handles complex PDFs (forms, tables, images)
-  More reliable with large documents
-  Active development and maintenance

### LLM Comparison:

**OpenAI GPT-4:**
-  Best quality responses
-  Faster inference
-  Larger context windows
-  Better reasoning capabilities
-  Requires API key and costs per token

**Local Ollama:**
-  Completely free
-  Full privacy (no data sent externally)
-  No API limits
-  Works offline
-  Slower inference
-  Requires good hardware

### Next Steps:

1. **Add More Documents**: Load PDFs with PyMuPDF, websites, or databases
2. **Custom Tools**: Create domain-specific tools
3. **Scale Up**: Use FAISS IVF or HNSW indices for millions of vectors
4. **Multi-Agent**: Build specialized agent teams
5. **MCP Server**: Deploy actual MCP servers
6. **Web Interface**: Add Gradio or Streamlit UI
7. **Fine-Tuning**: Train models on your data
8. **Production Deploy**: Containerize and deploy with FAISS persistence

In [ ]:
# Quick reference: Main components

print("\n" + "="*80)
print(" QUICK REFERENCE")
print("="*80 + "\n")

print("1️⃣  LLM Setup:")

print("    # OpenAI")
print("    llm = ChatOpenAI(model='gpt-4o', api_key=os.getenv('OPENAI_API_KEY'))")
print("    embeddings = OpenAIEmbeddings()")


print("\n2️⃣  Load PDFs (PyMuPDF):")
print("    documents = load_pdf_with_pymupdf('your_file.pdf')")
print("    # 3-5x faster than pypdf!")

print("\n3️⃣  FAISS Vector Store:")
print("    vectorstore = FAISS.from_documents(docs, embeddings)")
print("    # Lightning-fast similarity search")

print("\n4️⃣  Search:")
print("    results = vectorstore.similarity_search(query, k=3)")

print("\n5️⃣  Run Agent:")
print("    answer = run_agent('your question')")

print("\n6️⃣  Interactive:")
print("    interactive_agent_session()")

print("\n7️⃣  Save/Load FAISS:")
print("    vectorstore.save_local('faiss_index')")
print("    vectorstore = FAISS.load_local('faiss_index', embeddings)")


print("\n" + "="*80 + "\n")

provider = "OpenAI GPT-4"
print(f" All systems ready! Using {provider}")
print(" Vector Database: FAISS (Industry Standard)")
print(" PDF Processor: PyMuPDF (Fast & Reliable)")
print("\n🎉 Congratulations on building a complete AI agent!")